In [ ]:
ef bound_state_ldm(n, l, delta_lj, r, max_iter=80):
    """
    Find bound eigenenergy by log-derivative matching.
    Uses n* only for an initial bracket guess.
    """
    n_eff = n - delta_lj
    E0 = -1.0/(2.0*n_eff**2)

    # bracket around E0
    E_lo = 1.6*E0   # more negative
    E_hi = 0.6*E0   # less negative

    # match index near turning point (Veff ~ E0)
    Vef = V_eff(r, l)
    i_match = int(np.argmin(np.abs(Vef - E0)))
    i_match = max(20, min(len(r)-21, i_match))

    def mismatch(E):
        k2 = 2.0*(E - Vef)
        uo = numerov_outward(r, k2, l)
        ui = numerov_inward_bound(r, k2, E)
        ui *= (uo[i_match]/ui[i_match])
        return log_derivative(uo, r, i_match) - log_derivative(ui, r, i_match), uo, ui

    f_lo, _, _ = mismatch(E_lo)
    f_hi, _, _ = mismatch(E_hi)

    # widen until sign change
    for _ in range(60):
        if np.sign(f_lo) != np.sign(f_hi):
            break
        E_lo *= 1.2
        E_hi *= 0.8
        f_lo, _, _ = mismatch(E_lo)
        f_hi, _, _ = mismatch(E_hi)
    else:
        raise RuntimeError("Failed to bracket bound eigenenergy. Increase r_max, shrink h, or adjust r_min.")

    uo_best = ui_best = None
    for _ in range(max_iter):
        Em = 0.5*(E_lo + E_hi)
        f_m, uo, ui = mismatch(Em)
        uo_best, ui_best = uo, ui
        if np.sign(f_m) == np.sign(f_lo):
            E_lo, f_lo = Em, f_m
        else:
            E_hi, f_hi = Em, f_m
        if abs(E_hi - E_lo) < 1e-12:
            break

    # stitch
    u = np.zeros_like(r)
    u[:i_match+1] = uo_best[:i_match+1]
    u[i_match+1:] = ui_best[i_match+1:]

    # normalize bound: ∫|u|^2 dr = 1
    norm2 = integrate.simpson(u * u, r)
    if not np.isfinite(norm2) or norm2 <= 0:
        raise RuntimeError(f"Wavefunction normalization failed: norm2={norm2}")
    u /= np.sqrt(norm2)
    # u /= np.sqrt(integrate.simpson(u*u, r))
    return Em, u

In [ ]:
# quantum defect for Cs nP3/2 (use your value; this is the common one)
if (l, j) == (1, 1.5):
    delta = 3.5590676
else:
    raise NotImplementedError("Add quantum defect for your (l,j).")

r = np.arange(r_min, r_max + h, h)

# bound state
E_bound, u_bound = bound_state_ldm(n, l, delta, r)

# photon / continuum energy
nu = constants.c / wavelength_m
omega_SI = 2.0*np.pi*nu
Eph_J = constants.h*nu
Eph_au = Eph_J * J_to_au

E_free = Eph_au - abs(E_bound)
if E_free <= 0:
    raise ValueError("Photon energy below threshold.")